<a href="https://colab.research.google.com/github/Yassmina-Abdo/pyspark_course/blob/main/spam_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [1]:
!pip install pyspark --q

     |████████████████████████████████| 281.3 MB 32 kB/s 
     |████████████████████████████████| 199 kB 50.7 MB/s 


In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[1]").appName('final').getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [6]:
df = spark.read.csv('/content/SMSSpamCollection.csv', inferSchema=True,sep='\t')

### Print the schema

In [7]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



### Rename the first column to 'class' and second column to 'text'

In [12]:
df=df.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [13]:
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [14]:
df.show(10)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



In [15]:
df.show(10,truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [16]:
from pyspark.sql.functions import *

In [18]:
df= df.withColumn("length",length(col("text")))

### Show the new dataframe

In [19]:
df.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [30]:
df.groupBy("class").agg(avg('length').alias("Avg. Lenght")).show()

+-----+-----------------+
|class|      Avg. Lenght|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [91]:
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, IDF

In [104]:
#1.
tokenizer = Tokenizer(inputCol="text", outputCol="text_token")
#2.
remover = StopWordsRemover(inputCol="text_token", outputCol="text_sw")
#3.
cv = CountVectorizer(inputCol="text_sw", outputCol="text_cv",minDF=2.0)
#4.
idf = IDF(inputCol="text_cv", outputCol="text_idf")

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [93]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [94]:
stringindexer= StringIndexer(inputCol="class", outputCol="label")
vectorAssembler= VectorAssembler(inputCols = ["text_idf", "length"], outputCol = "features")

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [95]:
from pyspark.ml.classification import NaiveBayes

In [96]:
nb_model = NaiveBayes()

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [105]:
from pyspark.ml import Pipeline

In [106]:
pipeline = Pipeline(stages=[tokenizer, remover, cv,idf,stringindexer,vectorAssembler,nb_model])

### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [107]:
trainDF, testDF = df.randomSplit([0.7, 0.3], seed=42)

### Fit your Pipeline model to the training data

In [108]:
pl_Model=pipeline.fit(trainDF)

### Perform predictions on tests dataframe

In [109]:
predDF = pl_Model.transform(testDF)

### Print the schema of the prediction dataframe

In [110]:
predDF.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- text_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text_sw: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text_cv: vector (nullable = true)
 |-- text_idf: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [111]:
from pyspark.ml.evaluation import *

In [113]:
eval= MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label")
acc_res  = eval.evaluate(predDF, {eval.metricName:"f1"})
print("Accuracy = %0.2f" % acc_res)

Accuracy = 0.97


# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>